![sql](img/sql-logo.jpg)

# A beginner's guide to databases, SQL, & using them with `pandas`

**Scenario:** You are a data analyst for the Homeland Security, trying to create reports on the active airports world wide. The data you need to access is in a SQL database. YOu need to be able to query for the data in a database!

## Learning goals:
- Goal 1: Summarize the use case for sql in the data science skill set
- Goal 2: Define key sql terminology
- Goal 3: Get information about DB schema and table structure
- Goal 4: Use basic SQL commands:
    - Construct SQL queries
    - Use JOIN to merge tables along logical columns
    - Grouping Data with SQL
- Goal 5: Query data from pandas dataframes using SQL
- Goal 6: Convert SQL to pandas

## Goal 1: Summarize

[Netflix has a great article](https://medium.com/netflix-techblog/notebook-innovation-591ee3221233) describing three different data roles at their company, their different needs, and their toolsets.

![netflix](img/netflix-data-roles.jpeg)

Examining that graphic, SQL shows up as one of the tools of the _Data Engineer_ 

Data Engineers provide the essential data architecture services that make data science possible.

![hierarchy](img/ai-hierachy.png)

[Source: Monica Rogati’s fantastic Medium post “The AI Hierarchy of Needs”
](https://hackernoon.com/the-ai-hierarchy-of-needs-18f111fcc007)

![etl](img/etl.png)

[img source: Jeff Hammerbacher’s slide from UC Berkeley CS 194 course ](https://bcourses.berkeley.edu/courses/1377158/pages/cs-194-16-introduction-to-data-science-fall-2015)

### What is a Relational Database? 

![rdb](img/relational-dbms-model.png)
[reference for image ](https://www.studytonight.com/dbms/database-model.php)

### POPULAR RDBMS

- SQLite
- MySQL
- PostgreSql
- Oracle DB
- SQL Server

***
## Goal 2: Database terminology

### Relational Database Schema

![schema](img/MySQL_Schema_Music_Example.png)

[source of image](https://database.guide/what-is-a-database-schema/)

### Table columns view
<img src="img/columns.png" width=650>

### Terminology

- Database: Collection of tables
- Schema: Describes database (tables and columns) OR subgroups of databases
- Queries: Calling the databases
- Tables: Where data live
- Views: Shortcuts to subsets of tables OR combination of multiple tables (contingent of tables)
- Primary Keys: What defines a row (unique across all rows but can be repeated across columns) *index in Pandas
- Foreign Keys: Concepts or implementations that may or may not be seen in tables
- Indexes: Explains where an element is stored
- Functions & User Defined Functions (UDFs)

***
### SQLite

<img src="img/SQLite-Python.jpg" width=650>

"SQLite is a C library that provides a lightweight disk-based database that doesn’t require a separate server process and allows accessing the database using a nonstandard variant of the SQL query language. Some applications can use SQLite for internal data storage. It’s also possible to prototype an application using SQLite and then port the code to a larger database such as PostgreSQL or Oracle." - [sqlite documentation](https://docs.python.org/2/library/sqlite3.html)



## But what about connecting to database servers?

To connect to other database types there are many different libraries:
- Oracle: [cx_Oracle](https://oracle.github.io/python-cx_Oracle/)
- MySQL: [MySQL-python](https://github.com/farcepest/MySQLdb1)
- PostgreSQL: [Psycopg2](http://initd.org/psycopg/docs/)
- Microsoft SQL Server: [pymssql](http://www.pymssql.org/en/stable/)

These all implement [PEP 249: DB API v2](https://www.python.org/dev/peps/pep-0249/)

***
## Goal 3: Get going with sqlite!

In [1]:
import sqlite3

#### Load a DB object with `connect` and `cursor`

In [2]:
con = sqlite3.connect('flights.db')
cursor = con.cursor()

#### Use sqlite_master to find all the tables in the schema
Get the schema of a database from a db in sqlite

In [3]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('airports',), ('airlines',), ('routes',)]


In [4]:
cursor.description

(('name', None, None, None, None, None, None),)

In [5]:
cursor.execute("SELECT name, type FROM sqlite_master;")
print(cursor.fetchall())

[('airports', 'table'), ('ix_airports_index', 'index'), ('airlines', 'table'), ('ix_airlines_index', 'index'), ('routes', 'table'), ('ix_routes_index', 'index')]


In [6]:
cursor.execute("SELECT * FROM sqlite_master;")
results = cursor.fetchall()
for res in results:
    print(res)

('table', 'airports', 'airports', 2, 'CREATE TABLE airports (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [city] TEXT,\n  [country] TEXT,\n  [code] TEXT,\n  [icao] TEXT,\n  [latitude] TEXT,\n  [longitude] TEXT,\n  [altitude] TEXT,\n  [offset] TEXT,\n  [dst] TEXT,\n  [timezone] TEXT\n)')
('index', 'ix_airports_index', 'airports', 3, 'CREATE INDEX ix_airports_index ON airports ([index])')
('table', 'airlines', 'airlines', 945, 'CREATE TABLE airlines (\n[index] INTEGER,\n  [id] TEXT,\n  [name] TEXT,\n  [alias] TEXT,\n  [iata] TEXT,\n  [icao] TEXT,\n  [callsign] TEXT,\n  [country] TEXT,\n  [active] TEXT\n)')
('index', 'ix_airlines_index', 'airlines', 946, 'CREATE INDEX ix_airlines_index ON airlines ([index])')
('table', 'routes', 'routes', 1393, 'CREATE TABLE routes (\n[index] INTEGER,\n  [airline] TEXT,\n  [airline_id] TEXT,\n  [source] TEXT,\n  [source_id] TEXT,\n  [dest] TEXT,\n  [dest_id] TEXT,\n  [codeshare] TEXT,\n  [stops] TEXT,\n  [equipment] TEXT\n)')
('index', 'ix_routes_i

In [8]:
cursor.description

(('type', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('tbl_name', None, None, None, None, None, None),
 ('rootpage', None, None, None, None, None, None),
 ('sql', None, None, None, None, None, None))

#### Get information about one table

**A note about** `execute`<br>
Each time you use it, you reset the value of cursor

In [13]:
#as a is used to create an alias for table name, not necessary
cursor.execute("SELECT * FROM airports as a").fetchone()

(0,
 '1',
 'Goroka',
 'Goroka',
 'Papua New Guinea',
 'GKA',
 'AYGA',
 '-6.081689',
 '145.391881',
 '5282',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [14]:
cursor.fetchone()

(1,
 '2',
 'Madang',
 'Madang',
 'Papua New Guinea',
 'MAG',
 'AYMD',
 '-5.207083',
 '145.7887',
 '20',
 '10',
 'U',
 'Pacific/Port_Moresby')

In [15]:
cursor.fetchmany(4)

[(2,
  '3',
  'Mount Hagen',
  'Mount Hagen',
  'Papua New Guinea',
  'HGU',
  'AYMH',
  '-5.826789',
  '144.295861',
  '5388',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (3,
  '4',
  'Nadzab',
  'Nadzab',
  'Papua New Guinea',
  'LAE',
  'AYNZ',
  '-6.569828',
  '146.726242',
  '239',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (4,
  '5',
  'Port Moresby Jacksons Intl',
  'Port Moresby',
  'Papua New Guinea',
  'POM',
  'AYPY',
  '-9.443383',
  '147.22005',
  '146',
  '10',
  'U',
  'Pacific/Port_Moresby'),
 (5,
  '6',
  'Wewak Intl',
  'Wewak',
  'Papua New Guinea',
  'WWK',
  'AYWK',
  '-3.583828',
  '143.669186',
  '19',
  '10',
  'U',
  'Pacific/Port_Moresby')]

#### Use description

In [16]:
cursor.description

(('index', None, None, None, None, None, None),
 ('id', None, None, None, None, None, None),
 ('name', None, None, None, None, None, None),
 ('city', None, None, None, None, None, None),
 ('country', None, None, None, None, None, None),
 ('code', None, None, None, None, None, None),
 ('icao', None, None, None, None, None, None),
 ('latitude', None, None, None, None, None, None),
 ('longitude', None, None, None, None, None, None),
 ('altitude', None, None, None, None, None, None),
 ('offset', None, None, None, None, None, None),
 ('dst', None, None, None, None, None, None),
 ('timezone', None, None, None, None, None, None))

#### Or use `Pragma`
`Pragma` tool [link here](https://www.sqlite.org/pragma.html#pragma_table_info)

**output**<br>
`(column id, column name, data type, whether or not the column can be NULL, and the default value for the column)`

In [17]:
cursor.execute("PRAGMA table_info(airports)")
info = cursor.fetchall()
print(*info, sep = "\n") 

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'city', 'TEXT', 0, None, 0)
(4, 'country', 'TEXT', 0, None, 0)
(5, 'code', 'TEXT', 0, None, 0)
(6, 'icao', 'TEXT', 0, None, 0)
(7, 'latitude', 'TEXT', 0, None, 0)
(8, 'longitude', 'TEXT', 0, None, 0)
(9, 'altitude', 'TEXT', 0, None, 0)
(10, 'offset', 'TEXT', 0, None, 0)
(11, 'dst', 'TEXT', 0, None, 0)
(12, 'timezone', 'TEXT', 0, None, 0)


In [20]:
mylist = [1, 2]
print(mylist)
print(*mylist, sep = "\n")
print(*mylist, sep = ",")

[1, 2]
1
2
1,2


#### Making fetch happen

`.fetchall()` is how you get the query results out of the object.

You can also `.fetchone()` or `.fetchmany()`

**Task:** Get the descriptive data for airlines and routes tables

In [18]:
cursor.execute("PRAGMA table_info(airlines)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'id', 'TEXT', 0, None, 0)
(2, 'name', 'TEXT', 0, None, 0)
(3, 'alias', 'TEXT', 0, None, 0)
(4, 'iata', 'TEXT', 0, None, 0)
(5, 'icao', 'TEXT', 0, None, 0)
(6, 'callsign', 'TEXT', 0, None, 0)
(7, 'country', 'TEXT', 0, None, 0)
(8, 'active', 'TEXT', 0, None, 0)


In [21]:
#how many rows in airlines
cursor.execute("SELECT count(*) arline_count FROM airlines a;")
info = cursor.fetchall()
print(*info, sep = "\n")

(6048,)


In [19]:
cursor.execute("PRAGMA table_info(routes)")
info = cursor.fetchall()
print(*info, sep = "\n")

(0, 'index', 'INTEGER', 0, None, 0)
(1, 'airline', 'TEXT', 0, None, 0)
(2, 'airline_id', 'TEXT', 0, None, 0)
(3, 'source', 'TEXT', 0, None, 0)
(4, 'source_id', 'TEXT', 0, None, 0)
(5, 'dest', 'TEXT', 0, None, 0)
(6, 'dest_id', 'TEXT', 0, None, 0)
(7, 'codeshare', 'TEXT', 0, None, 0)
(8, 'stops', 'TEXT', 0, None, 0)
(9, 'equipment', 'TEXT', 0, None, 0)


***
## Goal 4: Use basic SQL commands 
- Construct SQL queries
- Use JOIN to merge tables along logical columns
- Grouping Data with SQL

### Construct SQL queries

**SELECT**: column names

**FROM**: joins

**WHERE**: condition

**GROUP BY**: column name

**ORDER BY**: sorting condition

**LIMIT**: number of rows returned

#### Options for each:

**SELECT**:  `DISTINCT` (unique), using `as` to rename columns, single number aggregates

**FROM:** also uses aliasing with `as`

**WHERE**: equals is only one `=`, `between`, `in`, wildcards `%`, `AND`, `OR`, `NOT`

**ORDER BY**: `asc` (default) and `desc`

**LIMIT**:  #

**Task**: 
- Select only active airlines in the UK from the airlines table

In [22]:
cursor.execute("""
SELECT name
FROM airlines
WHERE active = 'Y'
  AND country = 'United Kingdom';
""")
cursor.fetchmany(5)

[('Astraeus',),
 ('Air Southwest',),
 ('Aurigny Air Services',),
 ('Air Wales',),
 ('AD Aviation',)]

- Select the unique list of countries with airports

In [25]:
cursor.execute("PRAGMA table_info(airports)").fetchall()

[(0, 'index', 'INTEGER', 0, None, 0),
 (1, 'id', 'TEXT', 0, None, 0),
 (2, 'name', 'TEXT', 0, None, 0),
 (3, 'city', 'TEXT', 0, None, 0),
 (4, 'country', 'TEXT', 0, None, 0),
 (5, 'code', 'TEXT', 0, None, 0),
 (6, 'icao', 'TEXT', 0, None, 0),
 (7, 'latitude', 'TEXT', 0, None, 0),
 (8, 'longitude', 'TEXT', 0, None, 0),
 (9, 'altitude', 'TEXT', 0, None, 0),
 (10, 'offset', 'TEXT', 0, None, 0),
 (11, 'dst', 'TEXT', 0, None, 0),
 (12, 'timezone', 'TEXT', 0, None, 0)]

In [31]:
cursor.execute("""
SELECT country, count(*) AS airport_count
FROM airports
GROUP BY country
ORDER BY 2 DESC;
""").fetchmany(10)

[('United States', 1697),
 ('Canada', 435),
 ('Germany', 321),
 ('Australia', 263),
 ('Russia', 249),
 ('France', 233),
 ('China', 219),
 ('Brazil', 213),
 ('United Kingdom', 210),
 ('India', 140)]

### SQL Joins

SQL joins can be used to both **add** data to a table and **remove** data from a table. 

<img src="img/venn.png" width=550>

**Task** Write a query that will join the latitude and longitude data from the airports table to the information on the routes table

In [6]:
cursor.execute("""SELECT *
                FROM airports
                WHERE id IN ('2965', '2990')
                """)
print(cursor.fetchall(), sep='\n')

[(2886, '2965', 'Sochi', 'Sochi', 'Russia', 'AER', 'URSS', '43.449928', '39.956589', '89', '4', 'N', 'Europe/Moscow'), (2910, '2990', 'Kazan', 'Kazan', 'Russia', 'KZN', 'UWKD', '55.606186', '49.278728', '411', '4', 'N', 'Europe/Moscow')]


In [3]:
cursor.execute("""
SELECT *
FROM airports
""")
print(*cursor.description, sep='\n')

('index', None, None, None, None, None, None)
('id', None, None, None, None, None, None)
('name', None, None, None, None, None, None)
('city', None, None, None, None, None, None)
('country', None, None, None, None, None, None)
('code', None, None, None, None, None, None)
('icao', None, None, None, None, None, None)
('latitude', None, None, None, None, None, None)
('longitude', None, None, None, None, None, None)
('altitude', None, None, None, None, None, None)
('offset', None, None, None, None, None, None)
('dst', None, None, None, None, None, None)
('timezone', None, None, None, None, None, None)


In [9]:
cursor.execute("""SELECT
                    r.*,
                    a_source.latitude AS source_lat,
                    a_source.longitude AS source_lon,
                    a_dest.latitude AS dest_lat,
                    a_dest.longitude AS dest_long

                FROM routes AS r

                LEFT JOIN airports AS a_source
                ON r.source_id = a_source.id

                LEFT JOIN airports AS a_dest
                ON r.dest_id = a_dest.id

                LIMIT 1
                """)
print(cursor.fetchall(), sep='\n')

[(0, '2B', '410', 'AER', '2965', 'KZN', '2990', None, '0', 'CR2', '43.449928', '39.956589', '55.606186', '49.278728')]


### Grouping statements

Combines `select` and `group by` when you want aggregates by values

`select` `min(x)` ... `max()`, `sum()`, etc

`group by x`

**Task**<br>
- Which countries have the highest amount of active airlines?
- Which countries have the highest amount of inactive airlines?
- What about airports by timezones?

In [12]:
# Which countries have the highest amount of active airlines?
cursor.execute("""
                SELECT country, count(*) AS airline_count
                FROM airlines
                WHERE active = 'Y'
                GROUP BY country
                ORDER BY airline_count DESC
                LIMIT 5
                """)
print(*cursor.fetchall(), sep='\n')

('United States', 141)
('Russia', 72)
('United Kingdom', 40)
('Germany', 37)
('Canada', 34)


In [13]:
# Which countries have the highest amount of inactive airlines?
cursor.execute("""
                SELECT country, count(*) AS airline_count
                FROM airlines
                WHERE active = 'N'
                GROUP BY country
                ORDER BY airline_count DESC
                LIMIT 5
                """)
print(*cursor.fetchall(), sep='\n')

('United States', 939)
('Mexico', 427)
('United Kingdom', 367)
('Canada', 284)
('Russia', 158)


In [17]:
# What about airports by timezones?
cursor.execute("""
                SELECT timezone, count(*) as tz_count
                FROM airports
                GROUP BY timezone
                ORDER BY tz_count DESC
                LIMIT 5
                """)
print(*cursor.fetchall(), sep='\n')

('America/New_York', 628)
('America/Chicago', 373)
('Europe/Berlin', 319)
('America/Anchorage', 258)
('Europe/Paris', 232)


In [ ]:
# It's always a good idea to close our connections when we're done
cursor.close()
con.close()

## Goal 5: Using sql within pandas to filter

`.query()`

[query documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.query.html)

In [18]:
# Get data for an example
import pandas as pd
shelter_data=pd.read_csv('https://data.austintexas.gov/api/views/9t4d-g238/rows.csv?accessType=DOWNLOAD')
shelter_data.dtypes

Animal ID           object
Name                object
DateTime            object
MonthYear           object
Date of Birth       object
Outcome Type        object
Outcome Subtype     object
Animal Type         object
Sex upon Outcome    object
Age upon Outcome    object
Breed               object
Color               object
dtype: object

In [19]:
max_data = shelter_data.query('Name == "Max"')
max_data.head()
#vs
shelter_data[shelter_data['Name']=="Max"]

,Animal ID,Name,DateTime,MonthYear,Date of Birth,Outcome Type,Outcome Subtype,Animal Type,Sex upon Outcome,Age upon Outcome,Breed,Color
1144,A796351,Max,08/24/2019 12:03:00 PM,08/24/2019 12:03:00 PM,05/31/2010,Transfer,Partner,Cat,Neutered Male,9 years,Domestic Shorthair,Black/White
1174,A618405,Max,08/23/2019 06:38:00 PM,08/23/2019 06:38:00 PM,07/15/2010,Adoption,NaN,Dog,Neutered Male,9 years,American Pit Bull Terrier Mix,White/Blue
1223,A653408,Max,08/23/2019 11:33:00 AM,08/23/2019 11:33:00 AM,05/06/2009,Return to Owner,NaN,Dog,Neutered Male,10 years,American Bulldog Mix,White/Black
1371,A783700,Max,08/20/2019 06:36:00 PM,08/20/2019 06:36:00 PM,11/04/2017,Adoption,NaN,Dog,Neutered Male,1 year,Labrador Retriever Mix,Chocolate/White
2153,A744193,Max,08/08/2019 03:17:00 PM,08/08/2019 03:17:00 PM,09/27/2016,Adoption,Foster,Dog,Neutered Male,2 years,Cairn Terrier Mix,Tan/White
2199,A789119,Max,08/07/2019 06:56:00 PM,08/07/2019 06:56:00 PM,12/17/2014,Adoption,Foster,Dog,Neutered Male,4 years,Catahoula Mix,Red
2445,A801136,Max,08/04/2019 11:26:00 AM,08/04/2019 11:26:00 AM,03/22/2008,Rto-Adopt,NaN,Dog,Intact Male,11 years,Miniature Pinscher,Red
2622,A800872,Max,08/01/2019 04:37:00 PM,08/01/2019 04:37:00 PM,06/12/2019,Adoption,NaN,Dog,Neutered Male,1 month,German Shepherd Mix,Tan/Black
2780,A800791,Max,07/29/2019 06:07:00 PM,07/29/2019 06:07:00 PM,07/26/2017,Return to Owner,NaN,Dog,Intact Male,2 years,Pit Bull,Brown/White
2915,A788916,Max,07/27/2019 05:16:00 PM,07/27/2019 05:16:00 PM,11/26/2018,Return to Owner,NaN,Dog,Neutered Male,7 months,Pit Bull Mix,Brown Brindle/White


In [ ]:
shelter_data.rename(index=str, columns={"Animal Type": "animal_type"}, inplace = True)
test = shelter_data.query('animal_type == "Dog"')
test.head()

## Goal 6: Transfering from sqlite to pandas

In [21]:
pd_con = sqlite3.connect("flights.db")
df = pd.read_sql_query("select * from airports limit 5;", pd_con)
df

,index,id,name,city,country,code,icao,latitude,longitude,altitude,offset,dst,timezone
0,0,1,Goroka,Goroka,Papua New Guinea,GKA,AYGA,-6.081689,145.391881,5282,10,U,Pacific/Port_Moresby
1,1,2,Madang,Madang,Papua New Guinea,MAG,AYMD,-5.207083,145.7887,20,10,U,Pacific/Port_Moresby
2,2,3,Mount Hagen,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826789,144.295861,5388,10,U,Pacific/Port_Moresby
3,3,4,Nadzab,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569828,146.726242,239,10,U,Pacific/Port_Moresby
4,4,5,Port Moresby Jacksons Intl,Port Moresby,Papua New Guinea,POM,AYPY,-9.443383,147.22005,146,10,U,Pacific/Port_Moresby


**Task**: 
Convert one of the earlier queries in the lesson to a pandas data frame

In [22]:
# Which countries have the highest amount of active airports?
df = pd.read_sql_query("""
                SELECT country, count(*) AS airline_count
                FROM airlines
                WHERE active = 'Y'
                GROUP BY country
                ORDER BY airline_count DESC
                LIMIT 5;
""", pd_con)
df

,country,airline_count
0,United States,141
1,Russia,72
2,United Kingdom,40
3,Germany,37
4,Canada,34


In [23]:
pd_con.close()

## Integration

The [Chinook database](https://github.com/lerocha/chinook-database) is a sample database, representing a digital media store.

You need to create a query that can rank tracks in term of popularity.

The name of the database is `Chinook_Sqlite.sqlite`

Database information:<br>
- How many tables are in the database?
- What's the primary key of each table?
- What foreign keys join the tables together?
- If you had to draw a schema of how the tables are connected, what would it look like?

To answer the question:<br>
- What are the max and min dates in the Invoice table?
- What tables would you need to answer "what is your most popular track?"
- What values from each table?

In [ ]:
conn = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = conn.cursor()

In [ ]:
# How many tables are in the database?

# your code here

In [ ]:
# What's the primary key of each table?

# your code here

In [ ]:
# What foreign keys join the tables together?

# your code here
# hint: use "PRAGMA foreign_key_list()"


In [ ]:
# What are the max and min dates in the Invoice table?
cur.execute("""
-- your code here;
""").fetchall()

In [ ]:
# What tables would you need to answer "what is your most popular track?"


In [ ]:
# What values from each table?


In [ ]:
# Put it all together:
# You need to create a query that can rank tracks in term of popularity.

results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

In [ ]:
# Advanced: get the artist who sang the song!
results = cur.execute("""
-- your code here;
""").fetchall()

print(*results, sep='\n')

## Reflection

Please fill out today's [Exit Ticket](https://docs.google.com/forms/d/1dZg9MqBmLqJHWupgSBfvhicrVT1vyTejtaz3sXYsgGk/edit)!